In [1]:
import math
import pandas as pd
from pandas import Timestamp
import numpy as np
import glob
import os
import datetime as dt
from datetime import datetime
from pprint import pprint
from collections import OrderedDict

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display, HTML

from typing import List, Sequence, Tuple
from sklearn.preprocessing import StandardScaler

import stockanalibs

pd.set_option('display.max_columns', None)


In [2]:
ticker = stockanalibs.ticker
folder = "Intraday stocks" 

date_to_check = None
# date_to_check = '2019-03' # if 'None' (just comment this line) all dates to be analyzed and final merged CSV to be saved

In [3]:
pattern = os.path.join(folder, f"{ticker}_*.csv")
files = glob.glob(pattern)

if not files:
    raise FileNotFoundError(f"No files found for ticker {ticker} in {folder}")

# If there are multiple files, you might sort them or choose the first one.
files.sort()  # sorts alphabetically
file_to_read = files[0]

df = pd.read_csv(file_to_read, index_col=0, parse_dates=["datetime"])
df = df[['open', 'high', 'low', 'close', 'volume']]

ask_bid_spread = 0.03 # percent
spread_fraction = ask_bid_spread / 100.0
# Create 'ask' as close price plus the spread fraction and 'bid' as close price minus the spread fraction.
df['ask'] = round(df['close'] * (1 + spread_fraction),4)
df['bid'] = round(df['close'] * (1 - spread_fraction),4)

df

,open,high,low,close,volume,ask,bid
datetime,,,,,,,
2014-04-03 10:42:00,573.00,573.0000,573.000,573.0000,100.0,573.1719,572.8281
2014-04-03 10:43:00,573.00,573.0000,573.000,573.0000,100.0,573.1719,572.8281
2014-04-03 11:04:00,573.00,573.0000,573.000,573.0000,561.0,573.1719,572.8281
2014-04-03 11:05:00,573.00,573.0000,573.000,573.0000,231.0,573.1719,572.8281
2014-04-03 11:34:00,570.01,570.0100,570.010,570.0100,173.0,570.1810,569.8390
...,...,...,...,...,...,...,...
2025-06-18 23:55:00,173.90,173.9445,173.867,173.8681,3136.0,173.9203,173.8159
2025-06-18 23:56:00,173.82,173.9500,173.790,173.9000,183.0,173.9522,173.8478
2025-06-18 23:57:00,173.95,173.9500,173.860,173.8601,240.0,173.9123,173.8079


In [4]:
display(HTML("""
<style>
.output_scroll {
    overflow-y: visible !important;
    max-height: none !important;
}
</style>
"""))


In [5]:
def plot_trades(df, trades, buy_threshold, performance_stats, trade_color="green"):
    """
    Plots the overall close-price series plus trade intervals and two continuous signals,
    with the signals shown on a secondary y-axis.

    • The base trace (grey) plots the close-price series on the primary y-axis.
    • Trade traces (green by default) indicate the intervals for each trade from the original trade list.
    • A dotted blue line shows the raw normalized "signal" on a secondary y-axis.
    • A dashed red line shows the smooth normalized signal on the secondary y-axis.
    • A horizontal dotted line is drawn at the buy_threshold.
    • Additionally, areas between each buy and sell event determined by the new 
      "trade_action" field (buy=+1, sell=-1) are highlighted (in orange).
    • An update menu is added with two buttons:
         - "Hide Trades": Hides only the trade-specific traces.
         - "Show Trades": Makes all traces visible.

    Parameters:
      df : pd.DataFrame
          DataFrame with a datetime index and at least the columns "close", "signal_norm",
          "signal_smooth_adjusted", and "trade_action".
      trades : list
          A list of tuples, each in the form:
            ((buy_date, sell_date), (buy_price, sell_price), profit_pc).
      buy_threshold : float
          The threshold used for candidate buy detection (shown as a horizontal dotted line on the 
          secondary y-axis).
      performance_stats : dict, optional
          Dictionary containing performance metrics. If provided and if it contains keys
          "Trade Gains ($)" and "Trade Gains (%)" (each a list), they will be added to the
          trade annotations. 
      trade_color : str, optional
          The color to use for the original trade traces.
    """
    fig = go.Figure()
    
    # Trace 0: Base close-price trace.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['close'],
        mode='lines',
        line=dict(color='grey', width=1),
        name='Close Price',
        hoverinfo='x+y',
        hovertemplate="Date: %{x}<br>Close: %{y:.2f}<extra></extra>",
    ))
    
    # Trade traces: one per original trade.
    for i, trade in enumerate(trades):
        # Unpack the trade tuple: ((buy_date, sell_date), (buy_price, sell_price), profit_pc)
        (buy_date, sell_date), (_, _), trade_return = trade
        trade_df = df.loc[buy_date:sell_date]
        fig.add_trace(go.Scatter(
            x=trade_df.index,
            y=trade_df['close'],
            mode='lines+markers',
            line=dict(color=trade_color, width=3),
            marker=dict(size=4, color=trade_color),
            name=f"Trade {i+1}",
            hoveron='points',
            hovertemplate=f"Trade {i+1}: Return: {trade_return:.2f}%<extra></extra>",
            visible=True
        ))
        
    # --------------------------------------------------------------------
    # New Trade Action Highlights: using the 'trade_action' field.
    # Extract rows where trade_action is not zero.
    trade_events = df[df["trade_action"] != 0]["trade_action"]
    pairs = []
    prev_buy = None
    for timestamp, action in trade_events.items():
        if action == 1:   # Buy signal
            prev_buy = timestamp
        elif action == -1 and prev_buy is not None:
            pairs.append((prev_buy, timestamp))
            prev_buy = None
    # For each buy-sell pair, add a vertical shaded region with annotation.
    for j, (buy_ts, sell_ts) in enumerate(pairs):
        if (performance_stats is not None and 
            "Trade Gains ($)" in performance_stats and 
            "Trade Gains (%)" in performance_stats and 
            len(performance_stats["Trade Gains ($)"]) > j and 
            len(performance_stats["Trade Gains (%)"]) > j):
            ann_text = (f"TA Trade {j+1}<br>$: {performance_stats['Trade Gains ($)'][j]}<br>"
                        f"%: {performance_stats['Trade Gains (%)'][j]}")
        else:
            ann_text = f"TA Trade {j+1}"
            
        fig.add_vrect(
            x0=buy_ts, x1=sell_ts,
            fillcolor="orange", opacity=0.25,
            line_width=0,
            annotation_text=ann_text,
            annotation_position="top left",
            annotation_font_color="orange"
        )
    # --------------------------------------------------------------------
    
    # Raw Signal trace: Plot the normalized "signal" on a secondary y-axis.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['signal_norm'],
        mode='lines',
        line=dict(color='blue', width=2, dash='dot'),
        name='Signal (Normalized)',
        hovertemplate="Date: %{x}<br>Signal: %{y:.2f}<extra></extra>",
        visible=True,
        yaxis="y2"
    ))
    
    # Smooth Signal trace: Plot the smooth normalized signal on a secondary y-axis.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['signal_smooth_adjusted'],
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        name='Smooth Signal (Normalized)',
        hovertemplate="Date: %{x}<br>Smooth Signal: %{y:.2f}<extra></extra>",
        visible=True,
        yaxis="y2"
    ))
    
    # Add a horizontal dotted line for the buy_threshold (on secondary y-axis).
    fig.add_hline(y=buy_threshold, line=dict(color="purple", dash="dot"),
                  annotation_text="Buy Threshold", annotation_position="top left", yref="y2")
    
    # Total traces: 1 Base + n_trades (original trades) + 2 (for the signal traces).
    n_trades = len(trades)
    total_traces = 1 + n_trades + 2
    vis_show = [True] * total_traces  
    vis_hide = [True] + ["legendonly"] * n_trades + [True, True]
    
    fig.update_layout(
        updatemenus=[
            {
                "type": "buttons",
                "direction": "left",
                "buttons": [
                    {
                        "label": "Hide Trades",
                        "method": "update",
                        "args": [{"visible": vis_hide}],
                    },
                    {
                        "label": "Show Trades",
                        "method": "update",
                        "args": [{"visible": vis_show}],
                    },
                ],
                "pad": {"r": 10, "t": 10},
                "showactive": True,
                "x": 0.9,
                "xanchor": "left",
                "y": 1.1,
                "yanchor": "top",
            }
        ],
        hovermode="x unified",
        template="plotly_white",
        title="Close Price, Trade Intervals, and Signals",
        xaxis_title="Datetime",
        yaxis_title="Close Price",
        height=700,
        yaxis2=dict(
            title="Signal (Normalized)",
            overlaying="y",
            side="right",
            showgrid=False,
        )
    )
    
    fig.show()


In [6]:

def aggregate_performance(perf_list, round_digits=3):
    """
    Given a list of daily performance dictionaries, aggregate them as follows:
      - For numeric keys, sum their values and round the result.
      - For list keys (like "Trade Gains ($)" and "Trade Gains (%)"), replace them
        with a message indicating the number of trades.
    Then update:
      - "Strategy Profit Difference ($)" = "Final Net Value ($)" - "Buy & Hold Gain ($)"
      - "Strategy Improvement (%)"    = "Final Net Return (%)" - "Buy & Hold Return (%)"
    Returns the aggregated performance dictionary.
    """
    aggregated = {}
    keys = set()
    for perf in perf_list:
        if perf is not None:
            keys.update(perf.keys())
    for key in keys:
        if key in ["Trade Gains ($)", "Trade Gains (%)"]:
            temp_list = []
            for perf in perf_list:
                if perf and key in perf and isinstance(perf[key], list):
                    temp_list.extend(perf[key])
            aggregated[key] = f"{len(temp_list)} trades"
        else:
            total = 0
            for perf in perf_list:
                if perf and key in perf and isinstance(perf[key], (int, float)):
                    total += perf[key]
            aggregated[key] = round(total, round_digits)
    if "Final Net Value ($)" in aggregated and "Buy & Hold Gain ($)" in aggregated:
        aggregated["Strategy Profit Difference ($)"] = round(
            aggregated["Final Net Value ($)"] - aggregated["Buy & Hold Gain ($)"], round_digits)
    if "Final Net Return (%)" in aggregated and "Buy & Hold Return (%)" in aggregated:
        aggregated["Strategy Improvement (%)"] = round(
            aggregated["Final Net Return (%)"] - aggregated["Buy & Hold Return (%)"], round_digits)
    return aggregated

def saveDF_checkDAY(df, ticker, min_prof_thr, max_down_prop, gain_tightening_factor, 
                    smooth_win_sig, pre_entry_decay, buy_threshold, trailing_stop_thresh, 
                    merging_retracement_thr, merging_time_gap_thr, reference_gain,
                    keep_cols=["open", "high", "low", "close", "volume", "bid", "ask", 
                               "trade_action", "StrategyEarning", "EarningDiff", "signal_smooth_adjusted"], 
                    date_to_check=None):
    """
    Processes daily trading data for a given ticker.
    
    • If a full day (e.g. "2018-02-22") is provided, that day is processed as before.
    
    • If a month string (e.g. "2018-02") is provided, the function:
         – Determines all unique trading days (using df.index.normalize()) in that month.
         – For each trading day:
               * Executes Steps 1–4.
               * Prints the day's performance dictionary in a fixed order only once.
               * Displays the day's plot (which shows full trade details) non-blockingly, then closes it.
               * Appends the day's performance dictionary to a list (for aggregation).
         – After processing all days, aggregates the daily performance dictionaries so that:
               • Numeric values are summed & rounded.
               • "Strategy Profit Difference ($)" is calculated as (Final Net Value – Buy & Hold Gain).
               • "Strategy Improvement (%)" is calculated as (Final Net Return – Buy & Hold Return).
               • The raw trade lists are replaced by a message showing the total trade count.
         – Finally, prints the final monthly summary in a neatly formatted way.
    
    • If no date is provided, the function merges, cleans, and resamples the entire dataset.
         In this branch, extra print statements have been added for each sub-step.
    
    Note:
      – Smoothing is wrapped in try/except to catch errors like "min_periods 1 must be <= window 0."
      – When filtering for a specific day, the index is normalized (using only the date portion).
    """
    performance_list = []  # To collect daily performance dictionaries in month mode.
    
    def process_single_day(day_str):
        """Processes one trading day (given as a 'YYYY-MM-DD' string) and appends its performance."""
        print(f"\n=== Processing trading day {day_str} ===")
        day_dt = pd.to_datetime(day_str)
        # Filter the DataFrame to only the records for this day.
        day_df = df.loc[df.index.normalize() == day_dt].copy()
        if day_df.empty:
            print(f"No data found for {day_str}. Skipping.")
            return
        try:
            print(f"Step 1/5 for {day_str} => smooth_prepost_trading_data started at {datetime.now():%H:%M:%S}")
            try:
                day_df = stockanalibs.smooth_prepost_trading_data(
                    df=day_df,
                    regular_start=stockanalibs.regular_start,
                    regular_end=stockanalibs.regular_end,
                    red_pretr_win=stockanalibs.red_pretr_win)
            except ValueError as ve:
                if "min_periods" in str(ve) and "window 0" in str(ve):
                    print(f"Warning: smoothing failed for {day_str} (window=0). Using unsmoothed data.")
                else:
                    raise ve
            print(f"Step 1/5 for {day_str} => completed at {datetime.now():%H:%M:%S}")
            
            print(f"Step 2/5 for {day_str} => identify_trades_daily started at {datetime.now():%H:%M:%S}")
            results_by_day = stockanalibs.identify_trades_daily(
                df=day_df,
                min_prof_thr=min_prof_thr,
                max_down_prop=max_down_prop,
                gain_tightening_factor=gain_tightening_factor,
                regular_start_shifted=stockanalibs.regular_start_shifted,
                regular_end=stockanalibs.regular_end,
                merging_retracement_thr=merging_retracement_thr, 
                merging_time_gap_thr=merging_time_gap_thr,
                day_to_check=day_str)
            print(f"Step 2/5 for {day_str} => completed at {datetime.now():%H:%M:%S}")
            
            print(f"Step 3/5 for {day_str} => add_trade_signal_to_results started at {datetime.now():%H:%M:%S}")
            results_by_day = stockanalibs.add_trade_signal_to_results(
                results_by_day=results_by_day,
                min_prof_thr=min_prof_thr,
                regular_start=stockanalibs.regular_start,
                reference_gain=reference_gain,
                smooth_win_sig=smooth_win_sig,
                pre_entry_decay=pre_entry_decay,
                buy_threshold=buy_threshold,
                trailing_stop_thresh=trailing_stop_thresh,
                is_centered=stockanalibs.is_centered)
            print(f"Step 3/5 for {day_str} => completed at {datetime.now():%H:%M:%S}")
            
            print(f"Step 4/5 for {day_str} => simulate_trading started at {datetime.now():%H:%M:%S}")
            results_all = stockanalibs.simulate_trading(
                results_by_day=results_by_day,
                regular_start=stockanalibs.regular_start,
                regular_end=stockanalibs.regular_end,
                ticker=ticker)
            print(f"Step 4/5 for {day_str} => completed at {datetime.now():%H:%M:%S}")
    
            target_date = pd.to_datetime(day_str).date()
            results_to_check = None
            for key in results_all.keys():
                if key.date() == target_date:
                    results_to_check = results_all[key]
                    break
            if results_to_check is None:
                print(f"No trading results for {day_str} (maybe a weekend or holiday).")
                return
            df_to_check, trade_to_check, performance_to_check = results_to_check
            print(f"Step 5/5 Plotting trading results for {day_str}")
            plot_trades(
                df=df_to_check,
                trades=trade_to_check,
                buy_threshold=buy_threshold,
                performance_stats=performance_to_check)
            # Build an ordered dictionary for a fixed print order.
            order = ["Final Net Value ($)", "Buy & Hold Gain ($)", 
                     "Strategy Profit Difference ($)", "Final Net Return (%)", 
                     "Buy & Hold Return (%)", "Strategy Improvement (%)",
                     "Trade Gains ($)", "Trade Gains (%)"]
            ordered_perf = OrderedDict()
            for k in order:
                if k in performance_to_check:
                    if isinstance(performance_to_check[k], (int, float)):
                        ordered_perf[k] = round(performance_to_check[k], 3)
                    else:
                        ordered_perf[k] = performance_to_check[k]
            print("Daily Performance:")
            pprint(ordered_perf)
            plt.show(block=False)
            plt.close()
            performance_list.append(performance_to_check)
        except Exception as e:
            print(f"Error processing {day_str}: {e}")
            return

    # Main branch:
    if date_to_check is not None:
        if len(date_to_check) == 7:
            print(f"\n Month mode: Processing trading days in {date_to_check}")
            trading_days = sorted(
                df.loc[df.index.strftime('%Y-%m') == date_to_check].index.normalize().unique().astype(str)
            )
            if not trading_days:
                print(f"No trading days found in month {date_to_check}.")
                return None
            for day_str in trading_days:
                process_single_day(day_str)
            final_summary = aggregate_performance(performance_list)
            print(f"\nFinal Summary for month {date_to_check}:")
            pprint(final_summary)
            return None
        else:
            return process_single_day(date_to_check)
    else:
        # When date_to_check is None, process the entire dataset in one go,
        # with step-by-step prints.
        print(f"\n Merging, cleaning and resampling started at {datetime.now():%H:%M:%S}")
        print("Step A: Identifying trades on the entire dataset started.")
        results_by_day = stockanalibs.identify_trades_daily(
            df=df,
            min_prof_thr=min_prof_thr,
            max_down_prop=max_down_prop,
            gain_tightening_factor=gain_tightening_factor,
            regular_start_shifted=stockanalibs.regular_start_shifted,
            regular_end=stockanalibs.regular_end,
            merging_retracement_thr=merging_retracement_thr, 
            merging_time_gap_thr=merging_time_gap_thr,
            day_to_check=None)
        print("Step A: Completed identifying trades.")
        
        print("Step B: Adding trade signals started.")
        results_by_day = stockanalibs.add_trade_signal_to_results(
            results_by_day=results_by_day,
            min_prof_thr=min_prof_thr,
            regular_start=stockanalibs.regular_start,
            reference_gain=reference_gain,
            smooth_win_sig=smooth_win_sig,
            pre_entry_decay=pre_entry_decay,
            buy_threshold=buy_threshold,
            trailing_stop_thresh=trailing_stop_thresh,
            is_centered=stockanalibs.is_centered)
        print("Step B: Completed adding trade signals.")
        
        print("Step C: Simulating trading started.")
        results_all = stockanalibs.simulate_trading(
            results_by_day=results_by_day,
            regular_start=stockanalibs.regular_start,
            regular_end=stockanalibs.regular_end,
            ticker=ticker)
        print("Step C: Completed simulating trading.")
        
        print("Step D: Merging daily data frames started.")
        dfs = [result[0] for result in results_all.values()]
        merged = pd.concat(dfs)[keep_cols].sort_index()
        merged.index = pd.to_datetime(merged.index)
        n_before = merged.shape[0]
        merged = merged.dropna()
        n_after = merged.shape[0]
        print(f"Step D: Dropped {n_before - n_after} rows due to NA values.")
        
        print("Step E: Resampling each day's data started.")
        def resample_day(group):
            full_idx = pd.date_range(start=group.index.min(), end=group.index.max(), freq='min')
            return group.reindex(full_idx).ffill()
        merged = merged.groupby(merged.index.date, group_keys=False).apply(resample_day)
        print("Step E: Completed resampling.")
        
        merged.to_csv(f"dfs training/{ticker}.csv", index=True)
        print(f" Merging, cleaning and resampling completed at {datetime.now():%H:%M:%S}")
        return merged


In [7]:
min_prof_thr, max_down_prop, gain_tightening_factor, smooth_win_sig, pre_entry_decay, buy_threshold, trailing_stop_thresh, \
merging_retracement_thr, merging_time_gap_thr = stockanalibs.signal_parameters(ticker)

In [8]:
def compute_reference_gain(df):
    """
    Computes the reference gain as the average daily price range (difference between the high price and the low price) 
    """
    daily_range = df.groupby(df.index.date).apply(lambda x: x["high"].max() - x["low"].min())
    reference_gain = daily_range.mean()
    return reference_gain

reference_gain = compute_reference_gain(df)

In [9]:
df_fin = saveDF_checkDAY(
                        df=df,
                        ticker=ticker,
                        min_prof_thr=min_prof_thr, 
                        max_down_prop=max_down_prop, 
                        gain_tightening_factor=gain_tightening_factor,
                        smooth_win_sig=smooth_win_sig, 
                        pre_entry_decay=pre_entry_decay, 
                        buy_threshold=buy_threshold, 
                        trailing_stop_thresh=trailing_stop_thresh,
                        merging_retracement_thr=merging_retracement_thr, 
                        merging_time_gap_thr=merging_time_gap_thr,
                        reference_gain=reference_gain,
                        date_to_check=date_to_check
                        )

df_fin


 Merging, cleaning and resampling started at 17:32:33
Step A: Identifying trades on the entire dataset started.
Step A: Completed identifying trades.
Step B: Adding trade signals started.
Step B: Completed adding trade signals.
Step C: Simulating trading started.
Step C: Completed simulating trading.
Step D: Merging daily data frames started.
Step D: Dropped 57531 rows due to NA values.
Step E: Resampling each day's data started.
Step E: Completed resampling.
 Merging, cleaning and resampling completed at 17:45:32


,open,high,low,close,volume,bid,ask,trade_action,StrategyEarning,EarningDiff,signal_smooth_adjusted
2014-04-03 13:07:00,570.500,570.50,570.5000,570.50,321.0,570.3288,570.6712,0,0.0,0.000,0.071363
2014-04-03 13:08:00,570.500,570.50,570.5000,570.50,321.0,570.3288,570.6712,0,0.0,0.000,0.078887
2014-04-03 13:09:00,570.250,570.25,570.2500,570.25,298.0,570.0789,570.4211,0,0.0,0.000,0.087231
2014-04-03 13:10:00,570.250,570.25,570.2500,570.25,298.0,570.0789,570.4211,0,0.0,0.000,0.096444
2014-04-03 13:11:00,570.500,570.50,570.5000,570.50,100.0,570.3288,570.6712,0,0.0,0.000,0.106627
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-18 20:49:00,173.980,174.02,173.8900,173.99,1604.0,173.9378,174.0422,0,0.0,1.395,0.000000
2025-06-18 20:50:00,173.980,173.98,163.4137,173.92,524.0,173.8678,173.9722,0,0.0,1.465,0.000000
2025-06-18 20:51:00,173.970,174.00,173.8933,173.90,3898.0,173.8478,173.9522,0,0.0,1.485,0.000000
2025-06-18 20:52:00,173.908,174.00,173.8600,173.97,779.0,173.9178,174.0222,0,0.0,1.415,0.000000
